In [14]:
import numpy as np
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pydantic as pd
from typing import Tuple

import sys
sys.path.append('../../../')
from tidy3d import Simulation, PML

class App:
    
    def __init__(self, sim:Simulation):
        self.sim = sim
        self.app = self.make_app()

    def make_app(self):

        app = JupyterDash(__name__)
            
        pml_labels = ['PML (x)', 'PML (y)', 'PML (z)']
        initial_labels = [label for label, pml_on in zip(pml_labels, self.sim.pml_layers) if pml_on is not None]
        
        app.layout = html.Div([
            
            # size
            html.Br(),
            html.Label('Size [x,y,z] (um)'),
            html.Br(),            
            html.Div([
                dcc.Input(value=self.sim.size[0], type='number', id='size_x')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([

                dcc.Input(value=self.sim.size[1], type='number', id='size_y')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Input(value=self.sim.size[2], type='number', id='size_z')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Br(),

            
            # grid size
            html.Br(),
            html.Label('Grid Size [x,y,z] (um)'),
            html.Br(),            
            html.Div([
                dcc.Input(value=self.sim.grid_size[0], type='number', id='grid_size_x')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([

                dcc.Input(value=self.sim.grid_size[1], type='number', id='grid_size_y')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Input(value=self.sim.grid_size[2], type='number', id='grid_size_z')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Br(),

            
            # run time
            html.Br(),
            html.Label('Run Time (ps)'),
            html.Br(),
            dcc.Input(value=self.sim.run_time*1e12, type='number', id='run_time'),   
            html.Br(),


            # pml
            html.Br(),
            html.Label('PML Layers [x,y,z]'),
            html.Br(),
            dcc.Checklist(
                options=pml_labels,
                value=initial_labels,
                id='pml_checklist'
            ),
            html.Br(),


            # symmetry
            html.Br(),
            html.Label('Symmetry [x,y,z]'),
            html.Br(),            
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=self.sim.symmetry[0], id='symmetry_x')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=self.sim.symmetry[1], id='symmetry_y')
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=self.sim.symmetry[2], id='symmetry_z')
            ], style={'width': '15%', 'display': 'inline-block'}),
    
            # subpixel
            html.Br(),
            html.Label('Subpixel Smoothening'),
            html.Br(),
            dcc.Checklist(
                options=['On',],
                value=['On'] if self.sim.subpixel else [],
                id='subpixel'
            ),
            html.Br(),

            # shutoff
            html.Br(),
            html.Label('Field Decay Shutoff'),
            html.Br(),
            dcc.Input(value=self.sim.shutoff, type='number', id='shutoff'),   
            html.Br(),

            
            # courant
            html.Br(),
            html.Label('Courant Number'),
            html.Br(),
            dcc.Input(value=self.sim.courant, type='number', id='courant'),   
            html.Br(),

            
        ])

        @app.callback(
            Output('pml_checklist', 'value'),
            Input('pml_checklist', 'value')
        )
        def set_model(value):
            pml_on_list = [label in value for label in pml_labels]
            self.sim.pml_layers = [PML() if pml_on else None for pml_on in pml_on_list]
            # print([True if pml_on else None for pml_on in pml_on_list])
            print(f'sim.pml_layers set to {["PML" if pml else "None" for pml in pml_on_list]}')
            return value

        @app.callback(
            Output('run_time', 'value'),
            Input('run_time', 'value')
        )
        def set_run_time(value):
            new_value = max(0, value)
            new_value = value

            self.sim.run_time = new_value * 1e-12
            print(f'sim.run_time set to {new_value:.2f} ps')
            return new_value
                  
        return app
    
    def run(self):
    
        self.app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [15]:
sim = Simulation(size=(1,1,1), grid_size=(.01, .01, .01), run_time=1e-12, pml_layers=(PML(), PML(), None))
app = App(sim=sim)

[13:42:17] WARNING  No sources in simulation.                               ]8;id=485584;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py\simulation.py]8;;\:]8;id=776111;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py#366\366]8;;\

In [16]:
app.run()

sim.run_time set to 1.00 pssim.pml_layers set to ['PML', 'PML', 'PML']

sim.run_time set to 1.00 ps
sim.pml_layers set to ['PML', 'PML', 'PML']


In [4]:
app.sim

Simulation(center=(0.0, 0.0, 0.0), type='Simulation', size=(1.0, 1.0, 1.0), grid_size=(0.01, 0.01, 0.01), medium=Medium(name=None, frequency_range=None, type='Medium', permittivity=1.0, conductivity=0.0), run_time=1e-12, symmetry=(0, 0, 0), structures=[], sources=[], monitors=[], pml_layers=(PML(num_layers=12, parameters=PMLParams(sigma_order=3, sigma_min=0.0, sigma_max=1.5, type='PMLParams', kappa_order=3, kappa_min=1.0, kappa_max=3.0, alpha_order=1, alpha_min=0.0, alpha_max=0.0), type='PML'), PML(num_layers=12, parameters=PMLParams(sigma_order=3, sigma_min=0.0, sigma_max=1.5, type='PMLParams', kappa_order=3, kappa_min=1.0, kappa_max=3.0, alpha_order=1, alpha_min=0.0, alpha_max=0.0), type='PML'), PML(num_layers=0, parameters=PMLParams(sigma_order=3, sigma_min=0.0, sigma_max=1.5, type='PMLParams', kappa_order=3, kappa_min=1.0, kappa_max=3.0, alpha_order=1, alpha_min=0.0, alpha_max=0.0), type='PML')), shutoff=1e-05, subpixel=True, courant=0.9, version='1.1.1')

sim.run_time set to 1.00 ps
sim.pml_layers set to ['PML', 'PML', 'PML']
